In [17]:
import types, yaml, re
from pathlib import Path
from typing import Any, Dict, List, Optional
from mlx_lm.utils import load, save_config
from datasets import load_dataset
#from mlx_lm.utils import load, save_config

#model, tokenizer = load('mlx-community/Llama-3.2-1B-Instruct-bf16')

In [18]:
main_path = Path.cwd()
model_list_path = Path(main_path /'config' /'config.yaml')
with open(model_list_path, 'r') as file:
    model_list = yaml.load(file, yaml.SafeLoader)
model_list['models']


['Llama-3.2-1B-Instruct-bf16',
 'Josiefied-Qwen3-0.6B-abliterated-v1-bf16',
 'gemma-3-1b-it-qat-4bit']

In [ ]:
dataset_name = "MarkrAI/KoCommercial-Dataset"
#"lcw99/wikipedia-korean-20240501-1million-qna"
system_context = 'instruction'
user_context = 'input'
assistant_context = 'output'
model_name = model_list['models'][0]

current_path = Path.cwd()
dataset_path = Path(current_path /'data'/ dataset_name.split('/')[-1])
dataset_path.mkdir(parents=True, exist_ok=True)


In [ ]:
load_dataset(dataset_name, split='train[:1]')


In [ ]:
ds = model_list.get ('datasets',[])
if dataset_name not in ds:
    ds.append(dataset_name)
    model_list['datasets'] = ds
    with open(model_list_path, 'w') as file:
        yaml.dump(model_list, file)

In [ ]:

ds1 =load_dataset(dataset_name, split='train[:80%]')
ds2 =load_dataset(dataset_name, split='train[80%:-10%]')
ds3 =load_dataset(dataset_name, split='train[-10%:]')
# Define a function to apply the chat template
def apply_chat_template(example):
    messages = [
        {"role": "system", "content": example[system_context]},
        {"role": "user", "content": example[user_context]},
        {"role": "assistant", "content": example[assistant_context]}
    ]
    return {"messages":messages}

# Apply the chat template function to the dataset
nds1 = ds1.map(apply_chat_template)
nds2 = ds2.map(apply_chat_template)
nds3 = ds3.map(apply_chat_template)


nds1.to_json(str(dataset_path) + '/train.jsonl')
nds2.to_json(str(dataset_path) + '/test.jsonl')
nds3.to_json(str(dataset_path) + '/validation.jsonl')


In [19]:
dataset_name = "maywell/koVast"
system_context = 'instruction'
user_context = 'input'
assistant_context = 'output'
model_name = model_list['models'][0]

current_path = Path.cwd()
dataset_path = Path(current_path /'data'/ dataset_name.split('/')[-1])
dataset_path.mkdir(parents=True, exist_ok=True)


In [28]:
d = load_dataset(dataset_name)

In [30]:
d

DatasetDict({
    train: Dataset({
        features: ['conversations'],
        num_rows: 684579
    })
})

In [34]:
ds2['conversations']

[[{'from': 'user', 'value': 'Windows Mobile 6에서 크로스 플랫폼 개발의 가능성이 있어?'},
  {'from': 'gpt',
   'value': '네, 있습니다. 크로스 플랫폼 개발은 여러 플랫폼에서 동일한 소프트웨어를 개발할 수 있기 때문에 효율성과 비용을 절감할 수 있는 큰 가능성을 보입니다.'},
  {'from': 'user',
   'value': '그럼 Windows Mobile 6이 더 이상 지원되지 않는다는 점은 어떻게 해결할 수 있어?'},
  {'from': 'gpt',
   'value': 'Windows Mobile 6이 더 이상 지원되지 않는다는 점은 사실이지만, 이 플랫폼을 위해 개발하는 것에는 여전히 가치가 있습니다. 많은 사용자를 보유하고 있으며, 오픈소스 프로젝트와 같은 타사 솔루션을 사용하여 일부 어려움을 극복할 수 있습니다.'},
  {'from': 'user', 'value': '그럼 오픈소스 프로젝트는 어떤 한계가 있어?'},
  {'from': 'gpt',
   'value': '오픈소스 프로젝트는 종종 버그가 있거나 문서화가 부족할 수 있습니다. 또한, 이러한 프로젝트는 종종 지원이 중단되어 장기적으로 사용하기 어려울 수 있습니다.'},
  {'from': 'user', 'value': '그렇다면 어떻게 해야 하나?'},
  {'from': 'gpt',
   'value': '적절한 노력과 연구를 통해 이러한 솔루션을 사용하여 성공적인 크로스 플랫폼 애플리케이션을 개발할 수 있습니다. 가능한 모든 오픈소스 프로젝트를 조사하고, 필요에 맞는 프로젝트를 선택하며, 프로젝트를 사용하기 전에 철저히 테스트하고, 프로젝트가 지원되지 않을 경우를 대비하여 백업 계획을 가지는 것이 중요합니다.'}],
 [{'from': 'user', 'value': '자유의지가 있다고 생각해?'},
  {'from': 'gpt',
   'value': '네, 자유의지는 인간이 스스로 선택을 하고 결과에 책임을 

In [20]:
ds = model_list.get ('datasets',[])
if dataset_name not in ds:
    ds.append(dataset_name)
    model_list['datasets'] = ds
    with open(model_list_path, 'w') as file:
        yaml.dump(model_list, file)

In [25]:
nds1.remove_columns('conversations')

Dataset({
    features: ['messages'],
    num_rows: 547663
})

In [35]:

ds1 =load_dataset(dataset_name, split='train[:80%]')
ds2 =load_dataset(dataset_name, split='train[80%:-10%]')
ds3 =load_dataset(dataset_name, split='train[-10%:]')
# Define a function to apply the chat template
def apply_chat_template(example):
    messages = [
        {"role": "system", "content": "You are a helpful AI talking to human"},
        {"role": "user", "content":example['conversations'][0]['value']},
        {"role": "assistant", "content": example['conversations'][1]['value']}
    ]
    return {"messages":messages}

# Apply the chat template function to the dataset
nds1 = ds1.map(apply_chat_template)
nds2 = ds2.map(apply_chat_template)
nds3 = ds3.map(apply_chat_template)

nds1.remove_columns('conversations')
nds2.remove_columns('conversations')
nds3.remove_columns('conversations')

nds1.to_json(str(dataset_path) + '/train.jsonl')
nds2.to_json(str(dataset_path) + '/test.jsonl')
nds3.to_json(str(dataset_path) + '/validation.jsonl')


Creating json from Arrow format: 100%|██████████| 69/69 [00:01<00:00, 46.14ba/s]


272225872